In [45]:
import json
import datetime
from typing import Any, Callable, Set, Dict, List, Optional
from pydantic import BaseModel, Field

from azure.ai.projects.models import (
    ResponseFormatJsonSchema,
    ResponseFormatJsonSchemaType,
)


def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

# Statically defined user functions for fast reference
user_functions: Set[Callable[..., Any]] = {
    fetch_weather,
}

In [46]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet, FunctionDefinition
# from user_functions import user_functions # user functions which can be found in a user_functions.py file.

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["AZURE_AI_PROJECT_CONN_STR"],
)

# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)


class AgentResponseForWeather(BaseModel):
    location: str = Field(description="The location for which weather information was requested")
    weather_raw_json: str = Field(description="The raw JSON string containing weather data")
    weather_verbatim: str = Field(description="The human-readable weather information extracted from the JSON")


agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="my-weather-agent",
    instructions="You are a weather bot. Use the provided functions to help answer questions.",
    toolset=toolset,
    response_format=ResponseFormatJsonSchemaType(
        json_schema=ResponseFormatJsonSchema(
            name="weather_report",
            description="Weather report for a specific location",
            schema=AgentResponseForWeather.model_json_schema(),
        )
    )
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_brN52V9Cgrt3P8mHGUqewEZk


In [47]:
functions.definitions[0]

{'type': 'function', 'function': {'name': 'fetch_weather', 'description': 'Fetches the weather information for the specified location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location to fetch weather for.'}}, 'required': ['location']}}}

In [48]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Hello, send an email with the datetime and weather information in New York?",
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_mFccOZ0wWKbmUw6k8Y5p0v9z
Created message, ID: msg_0rdB0pE5K7NCtN6FnwzaKIhs


In [49]:
thread.as_dict()

{'id': 'thread_mFccOZ0wWKbmUw6k8Y5p0v9z',
 'object': 'thread',
 'created_at': 1747204212,
 'metadata': {},
 'tool_resources': {}}

In [50]:
thread_messages = project_client.agents.list_messages(thread_id=thread.id, limit=10)

In [51]:
project_client.agents.get_message(
    thread_id=thread.id,
    message_id=message.id,)

{'id': 'msg_0rdB0pE5K7NCtN6FnwzaKIhs', 'object': 'thread.message', 'created_at': 1747204212, 'assistant_id': None, 'thread_id': 'thread_mFccOZ0wWKbmUw6k8Y5p0v9z', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, send an email with the datetime and weather information in New York?', 'annotations': []}}], 'attachments': [], 'metadata': {}}

In [ ]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# # Delete the agent when done
# project_client.agents.delete_agent(agent.id)
# print("Deleted agent")


# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

In [ ]:
for d in messages.data:
    # print(d)
    print(d.role)
    print(d.content[0].text)

MessageRole.AGENT
{'value': '{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25\\\\u00b0C\\"}","weather_verbatim":"The weather in New York is Sunny with a temperature of 25°C."}', 'annotations': []}
MessageRole.USER
{'value': 'Hello, send an email with the datetime and weather information in New York?', 'annotations': []}


In [ ]:
last_message = messages.get_last_message_by_role("assistant")

In [ ]:
last_message

{'id': 'msg_Ulu4TIwX9jY9K6q8oB3PedkM', 'object': 'thread.message', 'created_at': 1747201697, 'assistant_id': 'asst_0JWVDMnTIztOyzaj1f4gc76G', 'thread_id': 'thread_1dp0iZx6wj1RDOJL42A5mI4P', 'run_id': 'run_ew50Ue2yEMfODHRqzEYhnYHo', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': '{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25\\\\u00b0C\\"}","weather_verbatim":"The weather in New York remains sunny with a temperature of 25°C. This message cannot be sent directly to an email; kindly use your mail application to compose and send the email."}', 'annotations': []}}], 'attachments': [], 'metadata': {}}

In [ ]:
type(last_message.content)
type(last_message.content[0])

azure.ai.projects.models._models.MessageTextContent

In [ ]:
msg_text_content = last_message.content[0]

In [ ]:
msg_text_content.text['value']

'{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25\\\\u00b0C\\"}","weather_verbatim":"The weather in New York remains sunny with a temperature of 25°C. This message cannot be sent directly to an email; kindly use your mail application to compose and send the email."}'

In [ ]:
wreport = AgentResponseForWeather.model_validate_json(msg_text_content.text['value'])

In [ ]:
print(wreport.location)

New York


In [ ]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="send the message to name@email.com",
)

In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Run finished with status: completed
Messages: {'object': 'list', 'data': [{'id': 'msg_Ulu4TIwX9jY9K6q8oB3PedkM', 'object': 'thread.message', 'created_at': 1747201697, 'assistant_id': 'asst_0JWVDMnTIztOyzaj1f4gc76G', 'thread_id': 'thread_1dp0iZx6wj1RDOJL42A5mI4P', 'run_id': 'run_ew50Ue2yEMfODHRqzEYhnYHo', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': '{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25\\\\u00b0C\\"}","weather_verbatim":"The weather in New York remains sunny with a temperature of 25°C. This message cannot be sent directly to an email; kindly use your mail application to compose and send the email."}', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_BFaol3FZ8Qr0Ww4n8WX5L4o9', 'object': 'thread.message', 'created_at': 1747201695, 'assistant_id': None, 'thread_id': 'thread_1dp0iZx6wj1RDOJL42A5mI4P', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'send the message to name@email.co

In [ ]:
for d in messages.data:
    # print(d)
    print(d.role)
    print(d.content[0].text)

MessageRole.AGENT
{'value': '{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25\\\\u00b0C\\"}","weather_verbatim":"The weather in New York remains sunny with a temperature of 25°C. This message cannot be sent directly to an email; kindly use your mail application to compose and send the email."}', 'annotations': []}
MessageRole.USER
{'value': 'send the message to name@email.com', 'annotations': []}
MessageRole.AGENT
{'value': '{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25\\\\u00b0C\\"}","weather_verbatim":"The weather in New York is Sunny with a temperature of 25°C."}', 'annotations': []}
MessageRole.USER
{'value': 'Hello, send an email with the datetime and weather information in New York?', 'annotations': []}
